# DESCRIPTION
Experiment with MIT's AST (Audio Spectrogram Transformer) for UAV Classification. 

# NOTES
- add inference metric to sweep loop
- change schedular
- more idk

In [2]:
from AST_helper.util import AudioDataset, train_test_split_custom, save_model
from AST_helper.engine import sweep_train, inference_loop
from AST_helper.model import auto_extractor, custom_AST
from AST_helper.util import save_model # noqa: F401

import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
from torchinfo import summary

import wandb

device = "cuda" if torch.cuda.is_available() else "cpu"
display(device)

'cuda'

In [3]:
data_path = "C:/Users/Sidewinders/Research_notebooks/Drone_classification/Research/UAV_Dataset_9"
model_name = "MIT/ast-finetuned-audioset-10-10-0.4593"
BATCH_SIZE = 16
SEED = 42
NUM_CUDA_WORKERS = 0
NUM_CLASSES =  9 
EPOCHS = 7
PINNED_MEMORY = True
SHUFFLED = True
ACCUMULATION_STEPS = 3 # multiplies by batch size for large batch size effect.
SAVE_MODEL = False
PROJECT_NAME = "AST_Sweeps"

torch.cuda.empty_cache()
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

In [4]:
sweep_config = {
    "name": "Schedular",
    "method": "random",
    "metric": {"goal": "maximize", "name": "test_acc"},
    "parameters": {
        "learning_rate": {"distribution":"uniform","min": 0.0001, "max": 0.001},
        "batch_size": {"values": [4,8,16,32]},
        "epochs" : {"values" : [7]},
        "optimizer" : {"values" : ["adamW"]},
        "scheduler" : {"values" : ["PolynomialLR"]}

        }    
}
sweep_id = wandb.sweep(sweep_config, project=PROJECT_NAME)
sweep_count = 100


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: ange3go8
Sweep URL: https://wandb.ai/andberg9-self/AST_Sweeps/sweeps/ange3go8


In [5]:
def make(config):
    # Make the data
    feature_extractor = auto_extractor(model_name)

    dataset = AudioDataset(data_path, feature_extractor)
    train_subset, test_subset = train_test_split_custom(dataset, test_size=0.2)  # type: ignore


    train_loader = DataLoader(dataset=train_subset, 
                                         batch_size=config.batch_size,
                                         num_workers=NUM_CUDA_WORKERS,
                                         pin_memory=PINNED_MEMORY,
                                         shuffle=SHUFFLED)
    
    test_loader = DataLoader(dataset=test_subset,
                                        batch_size=config.batch_size, 
                                        num_workers=NUM_CUDA_WORKERS,
                                        pin_memory=PINNED_MEMORY,
                                        shuffle=SHUFFLED)
    
    # if inference_subset: # may not be defined
    #     inference_dataloader_custom = DataLoader(dataset=inference_subset,
    #                                     batch_size=config.batch_size, 
    #                                     num_workers=NUM_CUDA_WORKERS,
    #                                     pin_memory=PINNED_MEMORY,
    #                                     shuffle=SHUFFLED) 

    # Make the model
    model = custom_AST(model_name, NUM_CLASSES, device)

    # Make the loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(),
                              lr=config.learning_rate)
    
    
    scheduler = optim.lr_scheduler.PolynomialLR(optimizer) 

    return model, train_loader, test_loader, criterion, optimizer, scheduler

In [6]:
def model_pipeline(config=None):

    # tell wandb to get started
    with wandb.init(config):
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config
      # make the model, data, and optimization problem
      model, train_loader, test_loader, criterion, optimizer, scheduler = make(config)
      print(model)

      results = sweep_train(model,
                      train_dataloader=train_loader,
                      test_dataloader=test_loader,
                      optimizer=optimizer,
                      scheduler=scheduler,
                      loss_fn=criterion,
                      epochs=config.epochs, # type: ignore
                      device=device,
                      num_classes=NUM_CLASSES
                      # patience=TRAIN_PATIENCE)
                      )
      
      inference_loop(model=model,
               device=device,
               loss_fn=criterion,
               inference_loader= train_loader)

    return model, results

In [7]:
# model,result = model_pipeline(config)
wandb.agent(sweep_id, model_pipeline, count=sweep_count)

wandb: Agent Starting Run: x8ajx6v4 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 7
wandb: 	learning_rate: 0.0009319810259145844
wandb: 	optimizer: adamW
wandb: 	scheduler: PolynomialLR
wandb: Currently logged in as: andberg9 (andberg9-self). Use `wandb login --relogin` to force relogin


ASTForAudioClassification(
  (audio_spectrogram_transformer): ASTModel(
    (embeddings): ASTEmbeddings(
      (patch_embeddings): ASTPatchEmbeddings(
        (projection): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ASTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ASTLayer(
          (attention): ASTSdpaAttention(
            (attention): ASTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ASTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ASTIntermediate(
       

  0%|          | 0/7 [00:00<?, ?it/s]

c:\Users\Sidewinders\anaconda3\envs\Research\Lib\site-packages\transformers\models\audio_spectrogram_transformer\modeling_audio_spectrogram_transformer.py:187: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  context_layer = torch.nn.functional.scaled_dot_product_attention(


Epoch 1/7 | Train Loss: 1.9678 | Train Acc: 0.3302 | Train F1: 0.3231 | Test Loss: 1.5025 | Test Acc: 0.6573 | Test F1: 0.6406
Epoch 2/7 | Train Loss: 1.2797 | Train Acc: 0.7541 | Train F1: 0.7407 | Test Loss: 1.0364 | Test Acc: 0.8656 | Test F1: 0.8624
Epoch 3/7 | Train Loss: 0.9422 | Train Acc: 0.8356 | Train F1: 0.8370 | Test Loss: 0.8319 | Test Acc: 0.8823 | Test F1: 0.8801
Epoch 4/7 | Train Loss: 0.7778 | Train Acc: 0.8546 | Train F1: 0.8556 | Test Loss: 0.7199 | Test Acc: 0.9052 | Test F1: 0.9056
Epoch 5/7 | Train Loss: 0.6997 | Train Acc: 0.8750 | Train F1: 0.8719 | Test Loss: 0.6742 | Test Acc: 0.8948 | Test F1: 0.8949
Epoch 6/7 | Train Loss: 0.6797 | Train Acc: 0.8736 | Train F1: 0.8760 | Test Loss: 0.6760 | Test Acc: 0.9010 | Test F1: 0.8949
Epoch 7/7 | Train Loss: 0.6735 | Train Acc: 0.8791 | Train F1: 0.8760 | Test Loss: 0.6724 | Test Acc: 0.8979 | Test F1: 0.8949
Train time on cuda: 341.9821492000003


...

wandb: Agent Starting Run: 1lrf7jgi with config:
wandb: 	batch_size: 16
wandb: 	epochs: 7
wandb: 	learning_rate: 0.0007086596902234274
wandb: 	optimizer: adamW
wandb: 	scheduler: PolynomialLR


ASTForAudioClassification(
  (audio_spectrogram_transformer): ASTModel(
    (embeddings): ASTEmbeddings(
      (patch_embeddings): ASTPatchEmbeddings(
        (projection): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ASTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ASTLayer(
          (attention): ASTSdpaAttention(
            (attention): ASTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ASTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ASTIntermediate(
       

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch 1/7 | Train Loss: 1.7158 | Train Acc: 0.4625 | Train F1: 0.4613 | Test Loss: 1.1552 | Test Acc: 0.7448 | Test F1: 0.7258
Epoch 2/7 | Train Loss: 0.9615 | Train Acc: 0.7847 | Train F1: 0.7756 | Test Loss: 0.7968 | Test Acc: 0.8333 | Test F1: 0.8335
Epoch 3/7 | Train Loss: 0.6915 | Train Acc: 0.8431 | Train F1: 0.8411 | Test Loss: 0.6362 | Test Acc: 0.8646 | Test F1: 0.8699
Epoch 4/7 | Train Loss: 0.5846 | Train Acc: 0.8681 | Train F1: 0.8670 | Test Loss: 0.5556 | Test Acc: 0.8906 | Test F1: 0.8981
Epoch 5/7 | Train Loss: 0.5351 | Train Acc: 0.8861 | Train F1: 0.8866 | Test Loss: 0.5135 | Test Acc: 0.9010 | Test F1: 0.8933
Epoch 6/7 | Train Loss: 0.5186 | Train Acc: 0.8972 | Train F1: 0.8983 | Test Loss: 0.5470 | Test Acc: 0.8698 | Test F1: 0.8933
Epoch 7/7 | Train Loss: 0.5186 | Train Acc: 0.8972 | Train F1: 0.8983 | Test Loss: 0.5264 | Test Acc: 0.9010 | Test F1: 0.8933
Train time on cuda: 334.74230759999773
Inference Loss: 0.5186, Accuracy: 89.72%


epoch,▁▂▃▅▆▇█
inference_accuracy,▁
inference_loss,▁
test_acc,▁▅▆██▇█
test_f1,▁▅▇████
test_loss,█▄▂▁▁▁▁
test_precision,▁▅▆████
test_recall,▁▅▇████
train_acc,▁▆▇████
train_f1,▁▆▇▇███
train_loss,█▄▂▁▁▁▁


In [8]:
# inference_loop(model=model,
#                device=device,
#                loss_fn=loss_fn,
#                inference_loader= inference_dataloader_custom)



# if not multiple_runs and wandb_init:
#     wandb.finish()

In [9]:
# if SAVE_MODEL:
#     save_model(model=model,
#             target_dir="saved_models",
#             model_name="AST_classifier_true.pt")